#### Critères = présence ou absence + fréquence de mot-clés

### Modularisation

In [1]:
import re
import os
from docx import Document
from collections import Counter
from nltk.corpus import stopwords
import nltk
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Téléchargement des stopwords français si nécessaire
nltk.download('stopwords')
stop_fr = set(stopwords.words('french'))

def lire_docx(path):
    """Extrait le texte brut d'un fichier Word (.docx)."""
    doc = Document(path)
    full_text = [p.text for p in doc.paragraphs]
    return "\n".join(full_text)

# --- Nettoyage de texte ---
def nettoyer_texte(texte: str) -> str:
    """Met en minuscules et supprime caractères spéciaux."""
    texte = texte.lower()
    texte = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ0-9\s]", " ", texte)
    texte = re.sub(r"\s+", " ", texte)
    return texte.strip()

# --- Extraction des mots clés d'une offre (stop words exclus) ---
def extraire_mots_cles_offre(offre_text: str):
    """
    Retourne la liste des mots clés uniques de l'offre après nettoyage
    et suppression des stop words français.
    """
    texte = nettoyer_texte(offre_text)
    mots = texte.split()
    # Exclusion des stop words
    mots_cles = [mot for mot in mots if mot not in stop_fr]
    return list(set(mots_cles))  # mots uniques

# --- Score de correspondance CV vs offre ---
def score_cv_offre(cv_text: str, offre_text: str):
    """
    Calcule un score de correspondance CV vs offre basé à la fois sur la présence 
    et la fréquence des mots-clés de l'offre dans le CV.

    Contrairement à la version simple, cette fonction prend en compte plusieurs 
    occurrences d'un mot-clé dans le CV tout en limitant l'impact d'un mot très 
    répété grâce à `max_occurrences`.

    Algorithme :
    1. Nettoyage du texte du CV (minuscules, suppression de ponctuation, etc.).
    2. Séparation du texte en mots et comptage des occurrences de chaque mot via Counter.
    3. Extraction des mots-clés uniques de l'offre (après nettoyage).
    4. Pour chaque mot-clé de l'offre, ajouter au score le nombre d’occurrences 
       présentes dans le CV, limité par `max_occurrences` pour éviter qu’un mot
       unique répété 100 fois domine le score.
    5. Normalisation : le score total est divisé par le score maximum possible 
       (nombre de mots-clés * max_occurrences) pour obtenir un pourcentage.
    
    Arguments :
    - cv_text (str) : texte complet du CV.
    - offre_text (str) : texte complet de l'offre.
    - max_occurrences (int, optionnel) : nombre maximal d’occurrences par mot-clé
      comptabilisées pour le score (défaut 2).

    Retour :
    - score_pct (float) : pourcentage de correspondance entre le CV et l'offre, 
      basé sur la diversité et la fréquence des mots-clés.

    Exemple :
    >>> score_cv_frequence(cv_text, offre_text)
    62.5
    """
    cv_clean = nettoyer_texte(cv_text)
    cv_mots = set(cv_clean.split())

    mots_cles = extraire_mots_cles_offre(offre_text)
    # print("mots_cles:", mots_cles)

    nb_trouves = sum(1 for mot in mots_cles if mot in cv_mots)
    Score = nb_trouves / len(mots_cles) if mots_cles else 0

    return Score * 100  # Score en pourcentage

def score_cv_frequence(cv_text: str, offre_text: str, max_occurrences=2):
    """
    Score basé sur la présence et la fréquence des mots-clés contrairement à
    la fonction précedente.
    max_occurrences limite l'impact d'un mot répété.
    """
    cv_clean = nettoyer_texte(cv_text)
    cv_mots = cv_clean.split()
    cv_counts = Counter(cv_mots)

    mots_cles = extraire_mots_cles_offre(offre_text)

    score_total = 0
    for mot in mots_cles:
        score_total += min(cv_counts.get(mot, 0), max_occurrences)

    max_score = len(mots_cles) * max_occurrences
    score_pct = (score_total / max_score) * 100 if max_score > 0 else 0

    return score_pct

offre = """
Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, la DIDM fait face à un besoin croissant de données fiables. C’est pourquoi un nouveau poste est créé.
Vous viendrez compléter une équipe composée d’une Chargée d’études et développements à 50 % et d’un Responsable Etudes et Développements. Sous la responsabilité de ce dernier, vos missions seront les suivantes :
Construire des pipelines de données pour alimenter la BI et l’analytique.
Modéliser et structurer les flux, tables et schémas
Garantir la qualité, la fiabilité et la sécurité des données
Développer de nouveaux datasets pour la BI de la DIDM
Mettre en place des standards de développement et de bonnes pratiques
Assurer le support et la résolution des incidents sur votre périmètre...

Votre boîte à outils
Excellente maîtrise de SQL (Oracle) et solide expérience en R
Connaissances en Julia, Java ou Scala appréciées
Pratique des outils de versioning (Git, Bitbucket, Github)
Expérience avec un outil ETL, idéalement Talend
Une première approche de la dataviz (Tableau, QlikView) est un atout
"""

# ---------------------------
# 1) Lecture des fichiers
# ---------------------------

def lire_tous_les_cvs(folder_path):
    """Retourne un dict {nom_fichier: texte_du_CV} pour tout le dossier."""
    cvs = {}
    for f in os.listdir(folder_path):
        if f.lower().endswith(".docx"):
            cvs[f] = lire_docx(os.path.join(folder_path, f))
    return cvs


# ---------------------------
# 2) Scoring des CVs
# ---------------------------

def scorer_les_cvs(cvs_dict, offre):
    """Retourne un dict {nom_fichier: score}."""
    scores = {}
    for filename, cv_text in cvs_dict.items():
        score = score_cv_frequence(cv_text, offre)
        scores[filename] = score
        print(f"Score de correspondance pour {filename} : {score:.1f} %")
    return scores


# ---------------------------
# 3) Construction du DataFrame
# ---------------------------

def construire_df_scores(scores_dict):
    """Transforme les scores en DataFrame trié par score décroissant."""
    df = pd.DataFrame([
        {"Fichier": f, "Score": s} for f, s in scores_dict.items()
    ])
    return df.sort_values(by="Score", ascending=False).reset_index(drop=True)


# ---------------------------
# 4) Identifier le CV original pour chaque variante
# ---------------------------

def mapper_cv_originaux(df_scores, original_files):
    """Ajoute une colonne base_name indiquant à quel CV original appartient chaque fichier."""
    
    original_bases = [f.rsplit(".", 1)[0] for f in original_files]

    def find_base(filename):
        for base in original_bases:
            if filename.startswith(base):
                return base + ".docx"
        return filename

    df_scores["base_name"] = df_scores["Fichier"].apply(find_base)
    return df_scores


# ---------------------------
# 5) Calcul des gains (ou pertes)
# ---------------------------

def ajouter_gains(df_scores, original_files):
    """Ajoute original_score + gain_vs_original.
    Sorties :
        df_scores (pd.DataFrame) : Le dataframe avec les nouvelles colonnes
                                    original_score : score du CV original sans amélioration
                                    gain_vs_original : différence entre le score du CV original sans 
                                                        amélioration et le CV actuel (avec ou sans
                                                        amélioration)                                    
    """
    
    # Dictionnaire : CV original -> score original
    original_score_dict = (
        df_scores[df_scores["Fichier"].isin(original_files)]
        .set_index("Fichier")["Score"]
        .to_dict()
    )

    df_scores["original_score"] = df_scores["base_name"].map(original_score_dict)
    df_scores["gain_vs_original"] = df_scores["Score"] - df_scores["original_score"]

    return df_scores


# ---------------------------
# 6) Tri final
# ---------------------------

def trier_groupes(df_scores, original_files):
    """Trier par CV original d’abord, puis original en premier, puis score décroissant."""
    
    df_scores["is_original"] = df_scores["Fichier"].isin(original_files)

    df_sorted = df_scores.sort_values(
        by=["base_name", "is_original", "Score"],
        ascending=[True, False, False]
    ).reset_index(drop=True)

    return df_sorted.drop(columns="is_original")


# ---------------------------
# 7) Affichage propre
# ---------------------------

def afficher_groupes(df_sorted):
    """Affichage console lisible."""
    for base, group in df_sorted.groupby("base_name"):
        print(f"\n📄 {base}")
        for _, r in group.iterrows():
            gain = f"(+{r['gain_vs_original']:.2f})" if r['gain_vs_original'] > 0 else f"({r['gain_vs_original']:.2f})"
            print(f"   {r['Fichier']:<80} {r['Score']:6.2f} {gain}")


# ---------------------------
# 8) Pipeline complet
# ---------------------------

def pipeline_complet(folder, original_files, offre):
    cvs = lire_tous_les_cvs(folder)
    scores = scorer_les_cvs(cvs, offre)

    df = construire_df_scores(scores)
    df = mapper_cv_originaux(df, original_files)
    df = ajouter_gains(df, original_files)
    df = trier_groupes(df, original_files)

    afficher_groupes(df)

    return df


# ---------------------------
# 9) Exemple d'utilisation
# ---------------------------
original_files = [
    "NRJBI_CEC_CV_Senior.docx",
    "NRJBI_ERE_CV - 20250930.docx",
    "NRJBI_CV_EMO_202510_revisionElise.docx",
]

offre = """
Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, la DIDM fait face à un besoin croissant de données fiables. C’est pourquoi un nouveau poste est créé.
Vous viendrez compléter une équipe composée d’une Chargée d’études et développements à 50 % et d’un Responsable Etudes et Développements. Sous la responsabilité de ce dernier, vos missions seront les suivantes :
Construire des pipelines de données pour alimenter la BI et l’analytique.
Modéliser et structurer les flux, tables et schémas
Garantir la qualité, la fiabilité et la sécurité des données
Développer de nouveaux datasets pour la BI de la DIDM
Mettre en place des standards de développement et de bonnes pratiques
Assurer le support et la résolution des incidents sur votre périmètre...

Votre boîte à outils
Excellente maîtrise de SQL (Oracle) et solide expérience en R
Connaissances en Julia, Java ou Scala appréciées
Pratique des outils de versioning (Git, Bitbucket, Github)
Expérience avec un outil ETL, idéalement Talend
Une première approche de la dataviz (Tableau, QlikView) est un atout
"""

df_resultat = pipeline_complet(
    folder="CVs",
    original_files=original_files,
    offre=offre
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eupho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Score de correspondance pour CV - Laurent D._14_01_2024.docx : 33.0 %
Score de correspondance pour CV_AnaA_20250226.docx : 26.7 %
Score de correspondance pour CV_CSA_NRJBI_20251016.docx : 21.6 %
Score de correspondance pour NRJBI_CEC_CV_Senior.docx : 14.2 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx : 5.1 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx : 17.0 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx : 23.3 %
Score de correspondance pour NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx : 30.1 %
Score de correspondance pour NRJBI_CV_EMO_202510_revisionElise.docx : 19.9 %
Score de correspondance pour NRJBI_ERE_CV - 20250930.docx : 27.3 %
Score de correspondance pour NRJBI_ERE_CV - 20250930_data_engineer_gemini-2.5-flash-lite_enhanced.docx : 25.0 %
Score de correspondance pour NRJBI_ERE_CV - 20250930_data_engineer_gemin

In [2]:
df_resultat

,Fichier,Score,base_name,original_score,gain_vs_original
0,CV - Laurent D._14_01_2024.docx,32.954545,CV - Laurent D._14_01_2024.docx,NaN,NaN
1,CV_AnaA_20250226.docx,26.704545,CV_AnaA_20250226.docx,NaN,NaN
2,CV_CSA_NRJBI_20251016.docx,21.590909,CV_CSA_NRJBI_20251016.docx,NaN,NaN
3,NRJBI_CEC_CV_Senior.docx,14.204545,NRJBI_CEC_CV_Senior.docx,14.204545,0.000000
4,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-pro_enhanced.docx,30.113636,NRJBI_CEC_CV_Senior.docx,14.204545,15.909091
5,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash_enhanced.docx,23.295455,NRJBI_CEC_CV_Senior.docx,14.204545,9.090909
6,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.5-flash-lite_enhanced.docx,17.045455,NRJBI_CEC_CV_Senior.docx,14.204545,2.840909
7,NRJBI_CEC_CV_Senior_data_engineer_gemini-2.0-flash_enhanced.docx,5.113636,NRJBI_CEC_CV_Senior.docx,14.204545,-9.090909
8,NRJBI_CV_EMO_202510_revisionElise.docx,19.886364,NRJBI_CV_EMO_202510_revisionElise.docx,19.886364,0.000000
9,NRJBI_ERE_CV - 20250930.docx,27.272727,NRJBI_ERE_CV - 20250930.docx,27.272727,0.000000
